In [14]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras import layers, models

# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Normalize the input data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes=100)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=100)

# Create the CNN model
model = Sequential()
model.add(Conv2D(30, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(70, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(160, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(100, activation=None))  # Softmax activation for 100 classes

# Define Gumbel-Softmax activation function
def gumbel_softmax(x):
    gumbel_noise = -tf.math.log(-tf.math.log(tf.random.uniform(tf.shape(x), dtype=x.dtype)))
    logits = x + gumbel_noise
    return tf.nn.softmax(logits)

# Apply Gumbel-Softmax activation function to the last fully connected layer
model.add(layers.Lambda(gumbel_softmax))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=30, validation_data=(x_test, y_test))


Epoch 1/30
1563/1563 [==============================] - 11s 6ms/step - loss: nan - accuracy: 0.0353 - val_loss: 4.7312 - val_accuracy: 0.0760
Epoch 2/30
1563/1563 [==============================] - 9s 6ms/step - loss: 4.4757 - accuracy: 0.1030 - val_loss: 4.2600 - val_accuracy: 0.1313
Epoch 3/30
1563/1563 [==============================] - 8s 5ms/step - loss: nan - accuracy: 0.1539 - val_loss: 3.9296 - val_accuracy: 0.1784
Epoch 4/30
1563/1563 [==============================] - 8s 5ms/step - loss: nan - accuracy: 0.1957 - val_loss: nan - val_accuracy: 0.1839
Epoch 5/30
1563/1563 [==============================] - 8s 5ms/step - loss: nan - accuracy: 0.2254 - val_loss: 3.6666 - val_accuracy: 0.2230
Epoch 6/30
1563/1563 [==============================] - 8s 5ms/step - loss: nan - accuracy: 0.2469 - val_loss: 3.5773 - val_accuracy: 0.2374
Epoch 7/30
1563/1563 [==============================] - 8s 5ms/step - loss: nan - accuracy: 0.2719 - val_loss: 3.5018 - val_accuracy: 0.2516
Epoch 8/30
1

In [23]:

# Evaluate the model
y_pred = model.predict(x_test)
y_pred = tf.argmax(y_pred, axis=1).numpy()
# y_test = tf.argmax(y_test, axis=1).numpy()

# accuracy = model.evaluate(x_test, y_test)[1]
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

313/313 [==============================] - 1s 3ms/step
Precision: 0.3044
Recall: 0.3023
F1 Score: 0.2981
